# 开始！

## 加载、检查模型

默认, cobrapy 为 _沙门氏菌_ 和 _大肠杆菌_ 有打包好的模型, 和 _大肠杆菌_ 核心代谢的一个 "教科书" 模型. 为了加载测试模型, 输入

In [1]:
from __future__ import print_function

import cobra
import cobra.test

# "ecoli" and "salmonella" are also valid arguments
model = cobra.test.create_test_model("textbook")

cobrapy 模型的反应, 代谢物, 和基因属性是一种特殊的被称为 `cobra.DictList` 的列表, 列表中的每一项由 `cobra.Reaction`, `cobra.Metabolite` and `cobra.Gene` 对象分别组成.

In [2]:
print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

95
72
137


使用 [Jupyter notebook](https://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/) 时这类信息显示为列表。

In [3]:
model

Name,e_coli_core
Memory address,0x01116ea9e8
Number of metabolites,72
Number of reactions,95
Objective expression,-1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
Compartments,"cytosol, extracellular"


像一个普通列表一样, `DictList` 中的对象可用 index 取得. 比如说, 在模型中取第 30 个反应 (使用了 [0-indexing](https://en.wikipedia.org/wiki/Zero-based_numbering)，所以 index 是 29):

In [4]:
model.reactions[29]

Reaction identifier,EX_glu__L_e
Name,L-Glutamate exchange
Memory address,0x011b8643c8
Stoichiometry,glu__L_e --> L-Glutamate -->
GPR,
Lower bound,0.0
Upper bound,1000.0


额外地, 元素可以由它们的 `id`，使用 `DictList.get_by_id()` 函数取得. 比如说，为了取得细胞溶质 ATP 代谢物 (cytosolic atp metabolite，id 是 "atp_c"), 我们可以这样做：

In [5]:
model.metabolites.get_by_id("atp_c")

Metabolite identifier,atp_c
Name,ATP
Memory address,0x011b7f82b0
Formula,C10H12N5O13P3
Compartment,c
In 13 reaction(s),"PYK, GLNS, ATPS4r, SUCOAS, PPCK, GLNabc, ATPM, ACKr, Biomass_Ecoli_core, ADK1, PPS, PFK, PGK"


额外的一个加成是, 使用交互式 shell，诸如 IPython 的用户可以用 tab 补全来在列表中列举元素. 虽然对大部分的代码我们不太建议这么做，因为 id 中可能会有 "-" 这样的字符, 在交互式 prompt 中这很有用:

In [6]:
model.reactions.EX_glc__D_e.bounds

(-10.0, 1000.0)

## 反应

我们将考虑葡萄糖-6-磷酸异构酶 (glucose 6-phosphate isomerase) 反应, 它相互转换了葡萄糖6-磷酸 (glucose 6-phosphate) 和果糖6-磷酸 (fructose 6-phosphate). 这个反应的 id 在我们的测试模型中是 PGI.

In [7]:
pgi = model.reactions.get_by_id("PGI")
pgi

Reaction identifier,PGI
Name,glucose-6-phosphate isomerase
Memory address,0x011b886a90
Stoichiometry,g6p_c <=> f6p_c D-Glucose 6-phosphate <=> D-Fructose 6-phosphate
GPR,b4025
Lower bound,-1000.0
Upper bound,1000.0


我们可以看到全名和催化（？）为字符串的反应。

In [8]:
print(pgi.name)
print(pgi.reaction)

glucose-6-phosphate isomerase
g6p_c <=> f6p_c


我们也可以看到反应的上下限. 因为 `pgi.lower_bound` < 0, 而且 `pgi.upper_bound` > 0, `pgi` 是可逆的.

In [9]:
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)
print(pgi.reversibility)

-1000.0 < pgi < 1000.0
True


我们也可以保证这个反应已经质量配平. 这个函数会返回与质量守恒相悖的元素. 如果它返回空，那么反应已配平.

In [10]:
pgi.check_mass_balance()

{}

为了添加反应物, 我们传递一个 `dict`，带一个反应物对象和它的系数。

In [11]:
pgi.add_metabolites({model.metabolites.get_by_id("h_c"): -1})
pgi.reaction

'g6p_c + h_c <=> f6p_c'

这个反应不再配平了

In [11]:
pgi.check_mass_balance()

{'H': -1.0, 'charge': -1.0}

我们可以去掉这个反应物, 于是它又质量守恒啦.

In [12]:
pgi.subtract_metabolites({model.metabolites.get_by_id("h_c"): -1})
print(pgi.reaction)
print(pgi.check_mass_balance())

g6p_c <=> f6p_c
{}


我们也可以从字符串构建反应. 但是, 一定要小心，保证反应中的 id 和模型中的那些相符. 箭头的方向也被使用来更新上下限.

In [13]:
pgi.reaction = "g6p_c --> f6p_c + h_c + green_eggs + ham"

unknown metabolite 'green_eggs' created
unknown metabolite 'ham' created


In [14]:
pgi.reaction

'g6p_c --> f6p_c + green_eggs + h_c + ham'

In [15]:
pgi.reaction = "g6p_c <=> f6p_c"
pgi.reaction

'g6p_c <=> f6p_c'

## 代谢物

我们将细胞溶质 ATP (cytosolic atp) 作为我们的代谢物, 它在我们的测试模型中的 id 是 `"atp_c"`.

In [16]:
atp = model.metabolites.get_by_id("atp_c")
atp

Metabolite identifier,atp_c
Name,ATP
Memory address,0x011b7f82b0
Formula,C10H12N5O13P3
Compartment,c
In 13 reaction(s),"PYK, GLNS, ATPS4r, SUCOAS, PPCK, GLNabc, ATPM, ACKr, Biomass_Ecoli_core, ADK1, PPS, PFK, PGK"


我们可以直接以字符串打印代谢物名称和 compartment (在这里是胞质溶胶 (cytosol)).

In [17]:
print(atp.name)
print(atp.compartment)

ATP
c


我们可以看到 ATP 在我们的模型是是一个带电分子。

In [18]:
atp.charge

-4

我们也可以看到代谢物的化学方程式

In [19]:
print(atp.formula)

C10H12N5O13P3


反应的属性提供了一个所有使用当前代谢物的反应的 `frozenset`. 我们可以用它来统计所有使用 atp 的反应.

In [20]:
len(atp.reactions)

13

像葡萄糖6-磷酸 (glucose 6-phosphate) 参与的反应就少一点.

In [21]:
model.metabolites.get_by_id("g6p_c").reactions

frozenset({<Reaction G6PDH2r at 0x11b870c88>,
           <Reaction GLCpts at 0x11b870f98>,
           <Reaction PGI at 0x11b886a90>,
           <Reaction Biomass_Ecoli_core at 0x11b85a5f8>})

## 基因

`gene_reaction_rule` 是一个反应激活的基因要求的布尔表示，在 [Schellenberger et al 2011 Nature Protocols 6(9):1290-307](http://dx.doi.org/doi:10.1038/nprot.2011.308) 中描述.

GPR 为作为字符串的反应对象存储为 gene_reaction_rule.

In [22]:
gpr = pgi.gene_reaction_rule
gpr

'b4025'

相关的基因对象也存在. 这些对象由反应和模型追踪。

In [23]:
pgi.genes

frozenset({<Gene b4025 at 0x11b844cc0>})

In [24]:
pgi_gene = model.genes.get_by_id("b4025")
pgi_gene

Gene identifier,b4025
Name,pgi
Memory address,0x011b844cc0
Functional,True
In 1 reaction(s),PGI


每个基因记录了它催化的反应对象。

In [25]:
pgi_gene.reactions

frozenset({<Reaction PGI at 0x11b886a90>})

在必要时，修改 gene_reaction_rule 会创建新的基因对象，并且更新所有关系.

In [26]:
pgi.gene_reaction_rule = "(spam or eggs)"
pgi.genes

frozenset({<Gene spam at 0x11b850908>, <Gene eggs at 0x11b850eb8>})

In [27]:
pgi_gene.reactions

frozenset()

新建的基因也被加入模型中

In [28]:
model.genes.get_by_id("spam")

Gene identifier,spam
Name,
Memory address,0x011b850908
Functional,True
In 1 reaction(s),PGI


`delete_model_genes` 函数会估计 GPR，如果反应被敲除的话，它的上下限都会设置为 0. 使用 `cumulative_deletions` flag，这个函数可以存储已有的删除操作或者重置它们.

In [29]:
cobra.manipulation.delete_model_genes(
    model, ["spam"], cumulative_deletions=True)
print("after 1 KO: %4d < flux_PGI < %4d" % (pgi.lower_bound, pgi.upper_bound))

cobra.manipulation.delete_model_genes(
    model, ["eggs"], cumulative_deletions=True)
print("after 2 KO:  %4d < flux_PGI < %4d" % (pgi.lower_bound, pgi.upper_bound))

after 1 KO: -1000 < flux_PGI < 1000
after 2 KO:     0 < flux_PGI <    0


undelete_model_genes 能用来重置基因删除操作

In [30]:
cobra.manipulation.undelete_model_genes(model)
print(pgi.lower_bound, "< pgi <", pgi.upper_bound)

-1000 < pgi < 1000


## 通过模型作为上下文使改变可回溯

经常, 有人会想对模型细微改变然后评估影响. 比如, 我们可能想依序敲除所有反应, 然后看它们会对目标函数 (objective function) 带来什么改变. 一种方式是在敲除之前用 `model.copy()` 创建模型的一个新副本. 但是, 即使是小模型, 这也会很慢，因为模型是非常复杂的对象. 更好的办法是在处理下一次反应前进行敲除、优化和手动重置反应限度. 既然这种情况经常发生, cobrapy 允许我们使用模型作为上下文, 来自动回退改变.

In [31]:
model = cobra.test.create_test_model('textbook')
for reaction in model.reactions[:5]:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

ACALD blocked (bounds: (0, 0)), new growth rate 0.873922
ACALDt blocked (bounds: (0, 0)), new growth rate 0.873922
ACKr blocked (bounds: (0, 0)), new growth rate 0.873922
ACONTa blocked (bounds: (0, 0)), new growth rate -0.000000
ACONTb blocked (bounds: (0, 0)), new growth rate -0.000000


如果我们看一下被敲除的反应, 可以看到它们的上下限都被回退到原值了.

In [32]:
[reaction.bounds for reaction in model.reactions[:5]]

[(-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0),
 (-1000.0, 1000.0)]

嵌套上下文也是支持的。

In [33]:
print('original objective: ', model.objective.expression)
with model:
    model.objective = 'ATPM'
    print('print objective in first context:', model.objective.expression)
    with model:
        model.objective = 'ACALD'
        print('print objective in second context:', model.objective.expression)
    print('objective after exiting second context:',
          model.objective.expression)
print('back to original objective:', model.objective.expression)

original objective:  -1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core
print objective in first context: -1.0*ATPM_reverse_5b752 + 1.0*ATPM
print objective in second context: 1.0*ACALD - 1.0*ACALD_reverse_fda2b
objective after exiting second context: -1.0*ATPM_reverse_5b752 + 1.0*ATPM
back to original objective: -1.0*Biomass_Ecoli_core_reverse_2cdba + 1.0*Biomass_Ecoli_core


大部分会改变模型的方法，包括添加、删除反应和代谢物、设置目标，都支持这样做。支持的方法和函数会在相关文档中说明。

虽然不会有实际影响, 为了语法方便，也可以在上下文外用不同的名字指向模型，比如说

In [34]:
with model as inner:
    inner.reactions.PFK.knock_out